In [1]:
import pandas as pd

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

In [2]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to C:\Users\Mike
[nltk_data]     PK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
max_words = 500
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [4]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [5]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [6]:
tokens = word_tokenize(train_corpus)

In [7]:
tokens[0:10]

['alisachachkaич',
 'уезжаааааааать',
 '❤',
 'тожена',
 'хотеть',
 'уезжать',
 'rt',
 'galyginvadim',
 'ребята',
 'девчата']

In [8]:
tokens_filtered = [word for word in tokens if word.isalnum()]
tokens_filtered = [word for word in tokens if len(word)>3]

In [9]:

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [10]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}


In [11]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [12]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping 
import os

In [14]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

In [15]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=300, input_length=max_len))
model.add(Conv1D(300, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 35s 109ms/step - loss: 0.6525 - accuracy: 0.5964 - val_loss: 0.6208 - val_accuracy: 0.6388
Epoch 2/20
319/319 [==============================] - 35s 110ms/step - loss: 0.6147 - accuracy: 0.6478 - val_loss: 0.6191 - val_accuracy: 0.6420
Epoch 3/20
319/319 [==============================] - 34s 107ms/step - loss: 0.6028 - accuracy: 0.6605 - val_loss: 0.6174 - val_accuracy: 0.6430
Epoch 4/20
319/319 [==============================] - 34s 106ms/step - loss: 0.5937 - accuracy: 0.6643 - val_loss: 0.6204 - val_accuracy: 0.6410


In [18]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 25ms/step - loss: 0.6251 - accuracy: 0.6390


Test score: 0.6250770688056946
Test accuracy: 0.6390247941017151


In [19]:
path_to_glove_file =  "C:/Users/Mike PK/YandexDisk/GB/Введение в обработку естественного языка/Lesson7/data/model.txt"

In [20]:
embeddings_index = {}
with open(path_to_glove_file, encoding="utf-8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        word = word.split('_')[0]
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 195436 word vectors.


In [21]:
len(embeddings_index['ещё'])

300

In [23]:
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 470 words (29 misses)


In [24]:
embedding_layer = Embedding(
    input_dim= max_words,
    output_dim=embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [25]:
model2 = Sequential()
model2.add(embedding_layer)
model2.add(Conv1D(300, 3))
model2.add(Activation("relu"))
model2.add(GlobalMaxPool1D())
model2.add(Dense(10))
model2.add(Activation("relu"))
model2.add(Dense(num_classes))
model2.add(Activation('softmax'))

In [26]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model2.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
319/319 [==============================] - 30s 92ms/step - loss: 0.6494 - accuracy: 0.6077 - val_loss: 0.6259 - val_accuracy: 0.6317
Epoch 2/20
319/319 [==============================] - 30s 93ms/step - loss: 0.6116 - accuracy: 0.6500 - val_loss: 0.6241 - val_accuracy: 0.6352
Epoch 3/20
319/319 [==============================] - 29s 92ms/step - loss: 0.5943 - accuracy: 0.6629 - val_loss: 0.6240 - val_accuracy: 0.6359
Epoch 4/20
319/319 [==============================] - 29s 92ms/step - loss: 0.5762 - accuracy: 0.6759 - val_loss: 0.6354 - val_accuracy: 0.6331


In [28]:
score = model2.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 24ms/step - loss: 0.6365 - accuracy: 0.6315


Test score: 0.6364816427230835
Test accuracy: 0.631530225276947


Большой разницы межде предъобыченными эмбедингами, и обучающимися, судя по метрикам нет.